## 1 导入相关库

In [25]:
import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from torch.nn import functional as F

## 2 获取数据集

In [26]:
def get_dataset(path, batch_size=32, transform=None):
    """
    加载MNIST数据集并将其转换为DataLoader对象。
    :param path: 数据集路径
    :param batch_size: 批处理大小
    :param transform: 数据预处理
    :return: 训练集与测试集的DataLoader对象
    """
    if transform is None:
        transform = torchvision.transforms.Compose([  # 对图像进行预处理
            torchvision.transforms.ToTensor(),  # 将图片转换成张量
            torchvision.transforms.Normalize(mean=(0.5,), std=(0.5,))  # 对图像进行归一化处理
        ])

    # 训练集
    mnist_train = torchvision.datasets.MNIST(  # 加载MNIST数据集，如果本地没有会自动下载
        root=path, train=True, transform=transform, download=True)
    # 测试集
    mnist_test = torchvision.datasets.MNIST(
        root=path, train=False, transform=transform, download=True)

    # 创建dataloader对象
    train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [27]:
# 查看MNIST数据集
mnist_train = torchvision.datasets.MNIST(  # 加载MNIST数据集，如果本地没有会自动下载
        root='./data', train=True, download=True)
print(mnist_train[0][1])
mnist_train[0][0]

5


## 3 定义模型

In [28]:
class Model(nn.Module):  # 构建卷积神经网络
    def __init__(self):
        super(Model, self).__init__()
        # 输入通道，输出通道，卷积核大小，步长，填充
        self.cov1 = nn.Conv2d(1, 15, 3, stride=1, padding=1)
        self.cov2 = nn.Conv2d(15, 45, 3, stride=1, padding=1)
        # 池化层 核大小2*2，步长2
        self.maxpool1 = nn.MaxPool2d(2, stride=2)
        self.maxpool2 = nn.MaxPool2d(2, stride=2)
        # 线性层  输入参数与前面的卷积与池化层输出通道数有关
        self.lin1 = nn.Linear(49*45, 10)

    def forward(self, x):
        x = self.cov1(x)  # 第一层卷积 28→28
        x = self.maxpool1(x)  # 第一层池化 28→14
        x = torch.relu(x)  # 激活函数
        x = self.cov2(x)  # 第二层卷积 14→14
        x = self.maxpool2(x)  # 第二层池化 14→7
        x = torch.relu(x)  # 激活函数
        x = x.view(x.size(0), -1)  # 将特征展平 7*7→49
        x = self.lin1(x)  # 全连接层 49→10
        return x

## 4 定义准确率函数

In [29]:
def acc_test(loader, model, device):
    """
    计算模型在测试集上的准确率。
    :param loader: 测试集的DataLoader对象
    :param model: 模型对象
    :param device: 设备对象
    :return: 准确率
    """
    model.eval()  # 将模型设置为评估模式
    acc = 0  # 准确的个数
    all_ = 0  # 总个数
    with torch.no_grad():  # 不计算梯度
        for i, (x, y) in enumerate(loader):  # 获取输入与输出
            x = x.to(device)  # 将图片转换为一维张量
            y = y.to(device)
            pre = model(x)  # 预测
            pre = torch.argmax(pre, dim=1)  # 获取预测结果每行中的最大值的坐标
            all_ += len(pre)  # 记录数据总数
            acc += (pre == y).sum().item()  # 记录准确的个数
    return acc / all_  # 返回准确率

## 5 定义训练函数

In [30]:
def train(path, output_=10, batch_size=128, lr=0.01, device='cpu', epochs=1):
    """
    训练模型
    :param path: 数据存放路径
    :param output_: 输出层神经元个数
    :param lr: 学习率
    :param device: 训练设备
    :param epochs: 训练轮数
    :param batch_size 批量大小
    :return: 返回训练后的模型
    """
    # 损失函数设置为交叉熵损失
    lossFuction = torch.nn.CrossEntropyLoss()

    # 创建一个卷积神经网络的对象
    model = Model()

    # 创建优化器
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # 使用Adam优化器

    # 获取数据
    train_loader, test_loader = get_dataset(path, batch_size=batch_size)

    # 将模型移动到设备上
    model.to(device)

    # 模型设置为训练模式
    model.train()

    # 训练模型
    for epoch in range(epochs):
        all_loss=[]
        acc_=[]
        for i, (x, y) in enumerate(train_loader):  # 获取输入与输出
            x = x.to(device)  # 将图片转换移动到设备上
            # 将输出数据转换为one_hot编码并转换为32位浮点数并移动到设备上
            y = torch.tensor(F.one_hot(y, num_classes=output_), dtype=torch.float32).to(device)
            optimizer.zero_grad()  # 将优化器梯度置零
            pre = model(x)  # 预测数据
            loss = lossFuction(pre, y)  # 计算损失
            loss.backward()  # 反向传播
            optimizer.step()  # 梯度更新
            if (i + 1) % 10 == 0:
                all_loss.append(float(loss))
                with open('loss.txt','w',encoding='utf-8') as f:
                    f.write(str(all_loss))
                acc=acc_test(test_loader, model, device)
                acc_.append(acc)
                with open('acc.txt','w',encoding='utf-8') as f:
                    f.write(str(acc_))
                print('准确率： ',acc)
                model.train()
    return model


## 6 训练

In [32]:
model = train('./data',device='cuda')

C:\Users\30535\AppData\Local\Temp\ipykernel_15240\1565514881.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(F.one_hot(y, num_classes=output_), dtype=torch.float32).to(device)


准确率：  0.7956
准确率：  0.8726
准确率：  0.9195
准确率：  0.94
准确率：  0.9494
准确率：  0.9414
准确率：  0.9564
准确率：  0.9627
准确率：  0.9638
准确率：  0.9662
准确率：  0.9662
准确率：  0.9675
准确率：  0.9655
准确率：  0.9686
准确率：  0.9646
准确率：  0.9661
准确率：  0.971
准确率：  0.9706
准确率：  0.9699
准确率：  0.974
准确率：  0.9739
准确率：  0.9762
准确率：  0.9782
准确率：  0.9754
准确率：  0.9758
准确率：  0.9773
准确率：  0.9766
准确率：  0.974
准确率：  0.9741
准确率：  0.9795
准确率：  0.9795
准确率：  0.9824
准确率：  0.9754
准确率：  0.9772
准确率：  0.9791
准确率：  0.9803
准确率：  0.9728
准确率：  0.9781
准确率：  0.9807
准确率：  0.9811
准确率：  0.9807
准确率：  0.9783
准确率：  0.9809
准确率：  0.9801
准确率：  0.9821
准确率：  0.9811


## 7 loss与准确度

In [33]:
%matplotlib qt
import matplotlib.pyplot as plt

In [34]:
with open('loss.txt','r',encoding='utf-8') as f:
    data=f.read()
data=eval(data)
fig=plt.figure()
plt.plot([i*10 for i in range(len(data))],data)
plt.xlabel('batch_num')
plt.ylabel('loss')
fig.show()

In [35]:
with open('acc.txt','r',encoding='utf-8') as f:
    data=f.read()
data=eval(data)
fig=plt.figure()
plt.plot([i for i in range(len(data))],data)
plt.xlabel('batch_num')
plt.ylabel('acc')
fig.show()